# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop(columns= 'timestamp')

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [4]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [5]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [6]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
params = {'n_factors': [20,50,100],
         'reg_all': [0.02, .05, .1]}
gs_svd = GridSearchCV(SVD, param_grid=params, n_jobs= -1)
gs_svd.fit(data)

{'rmse': 0.8692583940177017, 'mae': 0.6689254665971289}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [8]:
# print out optimal parameters for SVD after GridSearch
print(gs_svd.best_score)
print(gs_svd.best_params)

{'rmse': 0.8692583940177017, 'mae': 0.6689254665971289}
{'rmse': {'n_factors': 50, 'reg_all': 0.05}, 'mae': {'n_factors': 50, 'reg_all': 0.05}}


In [9]:
# cross validating with KNNBasic
knn_basic= KNNBasic(sim_options={'name':'pearson',
                                'user_based': True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs= -1)

In [10]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
print('---------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97005949, 0.97798836, 0.97700151, 0.96925671, 0.96526588]))
('test_mae', array([0.75027203, 0.75413446, 0.75510373, 0.7505806 , 0.74347573]))
('fit_time', (0.6693861484527588, 0.8298013210296631, 0.7109279632568359, 0.8173291683197021, 0.7577581405639648))
('test_time', (1.494255542755127, 1.363776445388794, 1.5597217082977295, 1.454061508178711, 1.2847070693969727))
---------------
0.9719143914938113


In [11]:
# cross validating with KNNBaseline
knn_baseline= KNNBaseline(sim_options={'name': 'pearson',
                                      'user_based': True})
cvn_knn_baseline = cross_validate(knn_baseline, data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [12]:
# print out the average score for the test set
for i in cvn_knn_baseline.items():
    print(i)
np.mean(cvn_knn_baseline['test_rmse'])

('test_rmse', array([0.87415705, 0.87750643, 0.87568021, 0.86890856, 0.88526437]))
('test_mae', array([0.66843831, 0.67252926, 0.67002248, 0.66238995, 0.67376114]))
('fit_time', (0.9122133255004883, 0.9002666473388672, 0.8453490734100342, 0.8838095664978027, 0.8487858772277832))
('test_time', (1.9720735549926758, 1.8119792938232422, 1.945673942565918, 1.7865846157073975, 1.8475151062011719))


0.8763033250613107

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [13]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [14]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [15]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [16]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.1616177511349854, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [19]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list      
user_rating = movie_rater(df_movies, 4, 'Comedy')

      movieId                 title          genres
2208     2935  Lady Eve, The (1941)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
      movieId                                title          genres
9715   188751  Mamma Mia: Here We Go Again! (2018)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
2
      movieId                  title          genres
2243     2978  Three to Tango (1999)  Comedy|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3
      movieId                    title  \
2144     2857  Yellow Submarine (1968)   

                                          genres  
2144  Adventure|Animation|Comedy|Fantasy|Musical  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
     movieId                        title                   genres
968     1269  Arsenic and Old Lace (1944)  Comedy|Mystery|Thriller
How do you rate thi

In [20]:
# try out the new function here!

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [21]:
user_rating
[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [22]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating, ignore_index= True)
new_data = Dataset.load_from_df(new_ratings_df, reader)

In [24]:
# train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.5)
svd_.fit(new_data.build_full_trainset())

In [26]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [27]:
# order the predictions from highest to lowest rated
ranked_movies = sorted(list_of_movies, key=lambda x: x[1], reverse=True)

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [30]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  2582    Guess Who's Coming to Dinner (1967)
Name: title, dtype: object 

Recommendation #  2 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  3 :  841    Streetcar Named Desire, A (1951)
Name: title, dtype: object 

Recommendation #  4 :  9618    Three Billboards Outside Ebbing, Missouri (2017)
Name: title, dtype: object 

Recommendation #  5 :  680    Philadelphia Story, The (1940)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.